# Performance monitoring
This notebook can be used to monitor the performance of the No-Show model over time. Performance is calculated over the patient group whom were not called, but did have a prediction. 

### Install dependencies

In [ ]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import relplot as rp
import seaborn as sns
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from sqlalchemy import select

from noshow.config import CLINIC_CONFIG
from noshow.dashboard.connection import init_session
from noshow.database.models import ApiCallResponse, ApiPrediction
from noshow.features.feature_pipeline import create_features, select_feature_columns
from noshow.preprocessing.load_data import (
    load_appointment_csv,
    process_appointments,
    process_postal_codes,
)

### *Reminder: change the .env file to select the production database*

## Data exploration

### Extract patients who are not called
Extract from ApiPrediction and ApiCallResponse. Select patients who had an prediction for an appointment, but were not called.

In [ ]:
session_object = init_session()

with session_object() as session:
    predicted_patients = session.execute(
        select(
            ApiPrediction.start_time,
            ApiPrediction.prediction,
            ApiPrediction.id,
            ApiPrediction.patient_id,
            ApiPrediction.appointment_id,
            ApiCallResponse.call_status,
        )
        .outerjoin(ApiPrediction.callresponse_relation)
        .outerjoin(ApiPrediction.patient_relation)
    ).all()

df = pd.DataFrame(
    predicted_patients,
    columns=[
        "start_time",
        "prediction",
        "id",
        "patient_id",
        "appointment_id",
        "call_status",
    ],
)
df.head()

### Percentage and count of not-called patients

In [ ]:
df["start_time"] = pd.to_datetime(df["start_time"])
df["month"] = df["start_time"].dt.to_period("M")
all_months = pd.period_range(
    start=df["month"].min(), end=df["month"].max() - 1, freq="M"
)
df = df[df["month"].isin(all_months)]

monthly_total = df["month"].value_counts().sort_index()
monthly_missing = (
    df[df["call_status"].isna() | (df["call_status"] == "Niet gebeld")]["month"]
    .value_counts()
    .sort_index()
)
monthly_percent = (
    (monthly_missing / monthly_total * 100)
    .reindex(all_months, fill_value=0)
    .rename("percent_none_or_niet_gebeld")
    .reset_index()
    .rename(columns={"index": "month"})
)


In [ ]:
monthly_percent["month_str"] = monthly_percent["month"].astype(str)
monthly_missing = monthly_missing.reindex(all_months, fill_value=0)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax1.bar(
    monthly_percent["month_str"],
    monthly_percent["percent_none_or_niet_gebeld"],
    color="salmon",
)
ax1.set_title("Percentage of not-called patients", fontsize=12)
ax1.set_ylabel("Percentage", fontsize=10)
ax1.grid(axis="y", linestyle="--", alpha=0.7)

ax2.plot(monthly_percent["month_str"], monthly_missing.values, color="gray", marker="o")
ax2.set_title("Count of not-called patients", fontsize=12)
ax2.set_ylabel("Count", fontsize=10)
ax2.set_xlabel("Month", fontsize=10)
ax2.grid(axis="y", linestyle="--", alpha=0.7)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

### Prediction distribution

In [ ]:
df_not_called = df[
    df["call_status"].isnull() | (df["call_status"] == "Niet gebeld")
].copy()

predictions = df_not_called.groupby("month")["prediction"].apply(list)
sorted_months = sorted(predictions.index)
recent_months = sorted_months[-12:]
predictions = predictions.loc[recent_months]
most_recent_month = recent_months[-1]

plt.figure(figsize=(10, 5))
palette = sns.color_palette("tab10", n_colors=len(predictions) - 1)
palette_iter = iter(palette)

for month, preds in predictions.items():
    if month == most_recent_month:
        sns.kdeplot(
            preds,
            fill=True,
            alpha=0.2,
            bw_adjust=0.3,
            color="black",
            label=str(month),
            linewidth=2.5,
        )
    else:
        sns.kdeplot(
            preds,
            fill=True,
            alpha=0.05,
            bw_adjust=0.3,
            color=next(palette_iter),
            label=str(month),
            linewidth=1.5,
        )

plt.title("Prediction Distributions by Month (Last 12 Months)", fontsize=12)
plt.xlabel("Prediction Value", fontsize=10)
plt.ylabel("Density", fontsize=10)
plt.legend(title="Month", fontsize=8)
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


## Results performance monitoring

### Load SQL data

In [ ]:
# Make new export and save to poliafspraken_no_show.csv
# export_data()

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_no_show.csv")
appointments_df = process_appointments(appointments_df, CLINIC_CONFIG)
appointments_df.head()

### ROC and AUC scores

In [ ]:
predicted_patients = (
    df_not_called.groupby("month")["appointment_id"]
    .apply(list)
    .reset_index(name="appointment_ids")
)

all_postalcodes = process_postal_codes("../data/raw/NL.txt")
with open("../output/models/no_show_model_cv.pickle", "rb") as f:
    model = pickle.load(f)

roc_data = []

for month, appointment_ids in zip(
    predicted_patients["month"], predicted_patients["appointment_ids"], strict=True
):
    if month <= pd.Period("2024-12", freq="M"):
        continue

    appointment_ids = [int(app_id) for app_id in appointment_ids]
    filtered_df = appointments_df[
        appointments_df["APP_ID"].isin(appointment_ids)
    ].copy()
    print(f"Processing month: {month}, Appointments: {len(filtered_df)}")

    if filtered_df.empty:
        print(f"No matching appointments found in data for month: {month}, skipping...")
        continue

    appointments_features = create_features(filtered_df, all_postalcodes).pipe(
        select_feature_columns
    )

    featuretable = appointments_features
    featuretable["no_show"] = (
        featuretable["no_show"].replace({"no_show": "1", "show": "0"}).astype(int)
    )
    X, y = featuretable.drop(columns="no_show"), featuretable["no_show"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=0, shuffle=False
    )

    y_pred = model.predict_proba(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])
    auc_score = roc_auc_score(y_test, y_pred[:, 1])

    roc_data.append(
        {
            "month": month,
            "fpr": fpr,
            "tpr": tpr,
            "auc": auc_score,
            "y_pred": y_pred[:, 1],
            "y_true": y_test,
        }
    )


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

for entry in roc_data:
    month = entry["month"]
    fpr = entry["fpr"]
    tpr = entry["tpr"]
    auc_score = entry["auc"]
    ax1.plot(fpr, tpr, label=f"{month} (AUC = {auc_score:.2f})")

ax1.plot([0, 1], [0, 1], linestyle="--", color="gray", label="Random Classifier")
ax1.set_xlabel("False Positive Rate")
ax1.set_ylabel("True Positive Rate")
ax1.set_title("ROC Curves by Month")
ax1.legend(loc="lower right")
ax1.grid(True)

months = [entry["month"] for entry in roc_data]
auc_scores = [entry["auc"] for entry in roc_data]
months_str = [str(month) for month in months]

ax2.plot(months_str, auc_scores, marker="o", linestyle="-", color="blue")
ax2.set_title("AUC Score Trend Over Time")
ax2.set_xlabel("Month")
ax2.set_ylabel("AUC Score")
ax2.set_ylim(0.0, 1.0)
ax2.grid(True)
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()


### Reliability diagram and calibration errors

In [ ]:
roc_data_sorted = sorted(roc_data, key=lambda x: x["month"])

second_last_data = roc_data_sorted[-2]
print(f"Showing reliability diagram for month: {second_last_data['month']}")

fig, ax = rp.rel_diagram(second_last_data["y_pred"], second_last_data["y_true"])
fig.set_size_inches(7, 6)
fig.suptitle(f"Reliability Diagram - {second_last_data['month']}", fontsize=12)
ax.set_xlabel("Mean Predicted Probability", fontsize=11)
ax.set_ylabel("Fraction of Positives", fontsize=11)
ax.tick_params(axis="both", labelsize=9)
ax.legend(fontsize=9, loc="lower right")

for txt in ax.texts:
    txt.set_fontsize(12)

plt.tight_layout()
plt.show()


In [ ]:
months = []
calib_errors = []

for entry in roc_data_sorted:
    month = entry["month"]
    error = rp.smECE(entry["y_pred"], entry["y_true"])
    months.append(str(month))
    calib_errors.append(error)

plt.figure(figsize=(10, 5))
plt.plot(months, calib_errors, marker="o", linestyle="-", color="darkorange")

plt.title("Calibration Error (smECE) Over Time", fontsize=12)
plt.xlabel("Month", fontsize=10)
plt.ylabel("Calibration Error (smECE)", fontsize=10)
plt.tick_params(axis="both", labelsize=9)

plt.ylim(0, max(calib_errors) * 1.1)
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
